In [ ]:
#------------------------------------------------------------------------------
# 인구조사 데이터 이진 분류 ONLY CODE
#------------------------------------------------------------------------------

In [ ]:
#---------------------------------------
# 문제정의 
#---------------------------------------

In [ ]:
# 5만불 이상인지 아닌지 소득 예측

In [ ]:
#---------------------------------------
# 라이브러리 가져오기
#---------------------------------------

In [ ]:
# 판다스 라이브러리
import pandas as pd

# 데이터 불러오기
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [ ]:
#---------------------------------------
# EDA
#---------------------------------------
# 데이터 샘플
train.head(n)
train.tail(n)
train.sample(n)
# 데이터 크기
train.shape, test.shape
# 자료형 (타입)
train.info()
# 수치형 컬럼 통계값 확인(train)
train.describe()
# 수치형 컬럼 통계값 확인(test)
test.describe()
# 범주형 컬럼 통계값 확인 (train)
train.describe(include='O')
# 범주형 컬럼 통계값 확인 (test)
test.describe(include='object')
# 결측치 (train)
train.isnull().sum()
# 결측치 (test)
test.isnull().sum()
# label(target)별 개수 확인
train['income'].value_counts()

In [ ]:
#---------------------------------------
# 데이터 전처리
#---------------------------------------

In [ ]:
train.drop(columns=['id'], inplace=True)
test.drop(columns=['id'], inplace=True)

In [4]:
#---------------------
# 결측치 처리
#---------------------
# 결측치채우기[범주형]  - 최빈값
m = train['workclass'].mode()[0]
train['workclass'] = train['workclass'].fillna(m)
m = train['native.country'].mode()[0]
train['native.country'] = train['native.country'].fillna(m)
train.isnull().sum()

# 결측치채우기[범주형]  - 임의값('X')
train['occupation'] = train['occupation'].fillna('X')
train.isnull().sum()

# 결측치 채우기[범주형] - test
test['workclass'] = test['workclass'].fillna(train['workclass'].mode()[0])
test['native.country']  = test['native.country'].fillna(train['native.country'].mode()[0])
test['occupation'] = test['occupation'].fillna("X")

# 결측치 채우기[수치형] - 평균값
value = int(train['age'].mean())
print("평균값:", value)
train['age'] = train['age'].fillna(value)
test['age'] = test['age'].fillna(value)

# 결측치 채우기[수치형] - 중앙값
value = int(train['hours.per.week'].median())
print("중앙값:", value)
train['hours.per.week'] = train['hours.per.week'].fillna(value)
test['hours.per.week'] = test['hours.per.week'].fillna(value)


#---------------------
# 이상치 처리
#---------------------
# age가 1이상인 데이터만 살림
print(train.shape)
train = train[train['age']>0]
print(train.shape)

#사분위수 이상치 범위 [예정]

#---------------------
# 인코딩[범주형 -> 수치형]
#---------------------
# 라벨(정답,y값) 분리
y_train = train.pop("income")
#-----------
# 레이블인코딩
#----------

cols = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex','native.country']
cols
from sklearn.preprocessing import LabelEncoder

for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

train.head()

# #----------
# # 레이블인코딩(개선 - test/train fit 맞지 않는경우
# #----------
# for col in cols:
#     le = LabelEncoder()
#     le.fit(pd.concat([train[col],test[col]] ,axis=0)) # 행합치기
#     train[col] = le.transform(train[col])
#     test[col] = le.transform(test[col])
    

#----------
# y 인코딩
#----------
target = y_train.map({'<=50K':0, '>50K':1})
target.value_counts()


#---------------------
# id컬럼삭제
#---------------------



NameError: name 'train' is not defined

In [ ]:
#---------------------------------------
# 검증 데이터 분할
#---------------------------------------

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train,
                                                  y_train,
                                                  test_size=0.2,
                                                  random_state=0)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
#---------------------------------------
# 모델 학습
#---------------------------------------

In [ ]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
pred=rf.predict_proba(X_val)  
pred[:10]

# LightGBM
import lightgbm as lgb
lgbmc = lgb.LGBMClassifier(random_state=0, verbose=-1)
lgbmc.fit(X_train, y_train)
pred = lgbmc.predict_proba(X_val)


In [ ]:
#---------------------------------------
# 성능 평가
#---------------------------------------

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val, pred[:,1])
print('roc_auc:', roc_auc)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, pred)
print('accuracy_score:', accuracy)

from sklearn.metrics import f1_score
f1 = f1_score(y_val, pred, pos_label='>50K')
print('f1_score:', f1)

In [ ]:
#---------------------------------------
# 파일로 저장
#---------------------------------------

In [ ]:
pred = lgbmc.predict_proba(test)
pred

lgbmc.classes_

submit = pd.DataFrame({'pred':pred[:,1]})
submit.to_csv("result.csv", index=False)

pd.read_csv("result.csv")

In [ ]:
#---------------------------------------
# 학습결과 모델파일(model.pkl) 외부로 저장
#---------------------------------------

In [ ]:
import joblib

# 모델 저장
model = rf
joblib.dump(model, "model.pkl")